In [9]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

import matplotlib.pyplot as plt
import scipy.io
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

from sklearn.manifold import TSNE
import seaborn as sns

import pandas as pd
import xlsxwriter
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

import nlpaug.augmenter.word.context_word_embs as aug

tf.get_logger().setLevel('ERROR')

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
#read the data into a pandas dataframe
df = pd.read_csv("bert_data_withoutClassWord.csv")
print(df.shape)
df.head(5)

(7115, 3)


,Unnamed: 0,text,class
0,0,this article is about the herbivorous mammals....,Antelope
1,1,"one new world species, the pronghorn of north ...",Antelope
2,2,"the english word ""animal"" first appeared in 14...",Antelope
3,3,"the word talopus and calopus, from latin, came...",Antelope
4,4,animal are not a cladistic or taxonomically de...,Antelope


In [3]:
#Add the new column which gives a unique number to each of these labels 

j = 0
for i in df['class'].unique():
    df.loc[df['class'] == i, ['class_num']] = j
    j += 1

#checking the results 
df.head(50000)

,Unnamed: 0,text,class,class_num
0,0,this article is about the herbivorous mammals....,Antelope,0.0
1,1,"one new world species, the pronghorn of north ...",Antelope,0.0
2,2,"the english word ""animal"" first appeared in 14...",Antelope,0.0
3,3,"the word talopus and calopus, from latin, came...",Antelope,0.0
4,4,animal are not a cladistic or taxonomically de...,Antelope,0.0
...,...,...,...,...
7110,7110,technology to use sponges as mouth protection ...,dolphin,49.0
7111,7111,"pesticides, heavy metals, plastics, and other ...",dolphin,49.0
7112,7112,"hundreds of orcas, animals and other members o...",dolphin,49.0
7113,7113,captured orcas and animals are confined to tan...,dolphin,49.0


In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df.text, 
    df.class_num, 
    test_size=0.2, # 20% samples will go to test dataset
    random_state=2022,
    stratify=df.class_num
)

In [5]:
embbeding_size = 5000
clustering_count = 3
numof_embb = 2
att_size = embbeding_size * clustering_count

In [8]:
vectorizer = TfidfVectorizer(max_features = embbeding_size).fit(X_train)
        
X_train_vectorized = vectorizer.transform(X_train)
    
clf = RandomForestClassifier()
clf.fit(X_train_vectorized, y_train)
    
predictions = clf.predict(vectorizer.transform(X_test))

print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

         0.0       0.28      0.39      0.33        38
         1.0       0.42      0.35      0.38        23
         2.0       0.68      0.54      0.60        28
         3.0       0.39      0.54      0.45        37
         4.0       0.31      0.17      0.22        23
         5.0       0.29      0.54      0.37        37
         6.0       0.36      0.43      0.39        44
         7.0       0.38      0.58      0.46        31
         8.0       0.46      0.48      0.47        23
         9.0       0.50      0.36      0.42        25
        10.0       0.60      0.29      0.39        21
        11.0       0.42      0.25      0.31        20
        12.0       0.34      0.28      0.31        36
        13.0       0.73      0.66      0.69        29
        14.0       0.65      0.38      0.48        29
        15.0       0.54      0.52      0.53        27
        16.0       0.50      0.17      0.25        12
        17.0       0.27    

In [10]:
augmenter = aug.ContextualWordEmbsAug(model_path='bert-base-uncased', action="insert")

ModuleNotFoundError: Missed torch library. Install torch by following https://pytorch.org/get-started/locally/`